In [1]:
import nibabel as nib
import numpy as np
from scipy import stats
import xml.etree.ElementTree as xml
import pandas as pd
from nilearn.plotting import plot_surf_stat_map
from matplotlib import pyplot as plt
import ciftools_af as ct
import warnings
warnings.filterwarnings('ignore')


def extract_zone_convergence(sub, hemi, brain_structure):
    # load surf
    surf_raw = nib.load(f'{root_dir}{sub}/Structural/{sub}.{hemi}.midthickness_MSMAll.32k_fs_LR.surf.gii')
    surf = []
    surf.append(surf_raw.darrays[0].data)
    surf.append(surf_raw.darrays[1].data)
    vertices, triangles = surf#[surf_raw.darrays[i].data for i in range(2)]

    # load labels
    labels = nib.load(f'{root_dir}{sub}/Structural/{sub}.zone_prim.32k_fs_LR.dlabel.nii')
    zones = labels.get_fdata().squeeze()

    brain_model = [x for x in labels.header.get_index_map(1).brain_models if x.brain_structure==brain_structure][0]
    offset = brain_model.index_offset
    cortex = np.asarray(brain_model.vertex_indices[0:])
    

    z = np.zeros(vertices.shape[0])
    z[cortex] = zones[offset:offset+len(cortex)]
    # next step takes advantage of prod of 1,2,3 == 6.
    coords = np.argwhere(np.prod(z[triangles], axis=1) == 6.)

    # take more posterior node:
    trig_of_interest = np.argmin([vertices[triangles[coords[0]]][0][:,1].mean(), vertices[triangles[coords[1]]][0][:,1].mean()])
    nodes_of_interest = triangles[coords[trig_of_interest]][0]
    
    return nodes_of_interest


### Find the scalar value associated to a set of vetices and a their percentile in a specified structure
def get_scalar_pctile(cifti_scalar, vertices, brain_structure, scalar_row=0):
    
    # extract scalar values of all vertices, and the features of the brain structure from cifti
    all_scalars = np.array(cifti_scalar.get_fdata()[scalar_row])
    brain_model = [x for x in cifti_scalar.header.get_index_map(1).brain_models if x.brain_structure==brain_structure]
    offset = brain_model[0].index_offset
    count = brain_model[0].index_count
    vertex_indices = np.array(brain_model[0].vertex_indices)
    idx = np.array([i for i,x in enumerate(vertex_indices) if x in vertices])
    
    # get scalars and relative percentile
    vertex_scalars = all_scalars[offset+idx]
    scalar_pctiles = [stats.percentileofscore(all_scalars[offset:offset+count],scalar) for scalar in vertex_scalars]
    del cifti_scalar
    
    return vertex_scalars, scalar_pctiles


/home/fralberti/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
### VARIABLES TO SET BEFORE RUNNING
# directory containing subdirectories named fter subject IDs that contain the timeseries and surface files
root_dir = "/home/fralberti/Data/HCP/"
# directory where all intermediate files and the final output will be saved
output_dir = "/home/fralberti/Data/Output_misc/"
# list of IDs of subjects to include in the analyses
f = open(f'{root_dir}subj_IDs_200.txt', 'r')
subj_id = np.array(f.read().splitlines())
del f

In [3]:
### Obtain the gradient values corresponding to the convergence nodes and relative percentile
print("Extracting principal gradient and percentile of convergence nodes, and performing permutation \nCurrent subject:")

# pre-assign the output dataframe
gradientile_df = pd.DataFrame(columns=['ID_vtx','ID_grad','hemisphere',
                                         'vertex1','vertex2','vertex3',
                                         'mean','percentile'])
for subj_grad in subj_id:
    print(f'\t{subj_grad}')
    # load a subject's gradient1
    grads = nib.load(f'{root_dir}{subj_grad}/{subj_grad}.gcca_200.32k_fs_LR.dscalar.nii')
    zones = nib.load(f'{root_dir}/HCP_S1200_GroupAvg_v1/zones.watershed.dlabel.nii').get_fdata()
    # get gradient and percentile of convergence nodes of all subjs from the current grad       
    mask_LR = []
    
    for hemi in ['L','R']:           
        # extract hemisphere's gradient
        brain_structure = ['CIFTI_STRUCTURE_CORTEX_LEFT' if hemi=='L' else 'CIFTI_STRUCTURE_CORTEX_RIGHT'][0]       
        offset, count, vertex_indices = ct.struct_info(brain_structure, grads)
        # apply lateral parietal mask
        z = [2 if hemi=='L' else 3][0]
        zone_hemi = zones[20, offset:offset+count]
        mask = zone_hemi == z
        mask_LR.extend(mask)
        grad_zone = grads.get_fdata()[0, offset:offset+count][mask]

        for subj_vtx in subj_id: 
            # extract principal gradient position of intersection vtx
            vtx_of_interest = extract_zone_convergence(subj_vtx, hemi, brain_structure)
            vtx_of_interest = np.sort(vtx_of_interest.tolist(),axis=0)
            vtx_grad = np.median(grad_zone[np.isin(vertex_indices[mask],vtx_of_interest)])
            vtx_pctile = stats.percentileofscore(grad_zone, vtx_grad)
                        
            # update output dataframe
            gradientile_df = gradientile_df.append({'ID_vtx':subj_vtx,'ID_grad':subj_grad,'hemisphere':hemi,
                                                    'vertex1':vtx_of_interest[0],'vertex2':vtx_of_interest[1],'vertex3':vtx_of_interest[2],
                                                    'mean':vtx_grad,'percentile':vtx_pctile}, ignore_index=True)

    ct.save_dscalar(np.array(mask_LR,ndmin=2), grads, f'{root_dir}{subj_grad}/Structural/{subj_grad}.zone_2.32k_fs_LR.dscalar.nii')


del grads
# save output        
gradientile_df.to_csv(f'{output_dir}gradientiles_s6.csv',index=False)
print("Done!")

Extracting principal gradient and percentile of convergence nodes, and performing permutation 
Current subject:
	100206
	100307
	100408
	101006
	101107
	101309
	101915
	102008
	102109
	102311
	102513
	102614
	102715
	102816
	103010
	103111
	103212
	103414
	103515
	103818
	104416
	104820
	105014
	105115
	105216
	105620
	105923
	106016
	106521
	106824
	107018
	107321
	107422
	107725
	108020
	108121
	108222
	108323
	108525
	108828
	109123
	109325
	109830
	110007
	110411
	111211
	111312
	111413
	111514
	111716
	112314
	112516
	112920
	113215
	113316
	113619
	114217
	114419
	114621
	114823
	114924
	115017
	115219
	115320
	115724
	115825
	116524
	117021
	117122
	117324
	117930
	118023
	118124
	118225
	118528
	118831
	118932
	119025
	119126
	120111
	120212
	120414
	120515
	120717
	121416
	121618
	122317
	122620
	122822
	123117
	123420
	123521
	123723
	123824
	123925
	124220
	124422
	124624
	124826
	125222
	125424
	125525
	126426
	127226
	127327
	127630
	127731
	127832
	127933
	128026
	128127


In [48]:
# Perform simple permutation test
gradientile_df = pd.read_csv(f'{output_dir}gradientiles_s6.csv')
X = gradientile_df[gradientile_df.ID_vtx==gradientile_df.ID_grad].set_index('ID_grad')
Y = gradientile_df[gradientile_df.ID_vtx!=gradientile_df.ID_grad].groupby(['ID_grad','hemisphere']).agg('median').reset_index(level=1)
L_wsrt = stats.wilcoxon(X.loc[X.hemisphere=='L','percentile'],Y.loc[X.hemisphere=='L','percentile'])
R_wsrt = stats.wilcoxon(X.loc[X.hemisphere=='R','percentile'],Y.loc[X.hemisphere=='R','percentile'])
print('LH: ', L_wsrt,'\n','RH: ', R_wsrt)

LH:  WilcoxonResult(statistic=9058.5, pvalue=0.3265831903993619) 
 RH:  WilcoxonResult(statistic=8900.5, pvalue=0.1607395062160054)
